# Accessing ACME
### Ray Smith

In [27]:
# Selenium Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Datetime Imports
import datetime

In [18]:
# Settings
url = "https://acme.wisc.edu/tools/schedule/schedule.php"

login_secret = "login.secret"
duo_secret = "duo.secret"

options = Options()
# options.headless = True

In [19]:
# Get Secrets
with open(login_secret) as f:
    user = f.readline()[:-1]
    password = f.readline()

In [40]:
# Date conversion functions
# String form = date=YYYY-MM-DD

# URL String to Date
def string_to_date(string):
    parts = string.split('-')
    year = int(parts[0])
    month = int(parts[1])
    day = int(parts[2])
    date = datetime.date(year=year, month=month, day=day)
    return date

# Date to URL String
def date_to_string(date):
    year = date.year
    month = date.month
    day = date.day
    return str(year) + '-' + str(month) + '-' + str(day)

In [20]:
# Login to ACME
def acme_login():
    browser = webdriver.Chrome(options=options)
    browser.get(url)

    # Fill in username
    user_field = browser.find_element_by_id('j_username')
    user_field.send_keys(user)

    # Fill in password
    pass_field = browser.find_element_by_id('j_password')
    pass_field.send_keys(password)

    # Click login
    login_button = browser.find_element_by_name('_eventId_proceed')
    login_button.click()
    
    # Return ACME pointed browser
    return browser

In [53]:
# Get Schedule Table by Date (as string)
def get_schedule_by_date(browser, date):
    new_url = url + '?date=' + date
    browser.get(new_url)
    
    table = browser.find_element_by_id('sch_table_verticle')
    return table

In [48]:
# Login
browser = acme_login()

In [54]:
# Access certain date
date = '2020-10-01'
table = get_schedule_by_date(browser, date)
table

<selenium.webdriver.remote.webelement.WebElement (session="6aa8d84e229c3ef2618937f9f3dfc8f0", element="39c8a9f2-f924-4cbb-8038-56a782914471")>

In [47]:
# Close browser
browser.close()